<a href="https://colab.research.google.com/github/Vakhranev/Gramota/blob/main/%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%81%D1%82%D1%80%D0%BE%D0%BA%20%D0%B8%D0%B7%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

# Настраиваем доступ к Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Открываем таблицу по URL
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1bVNlA7qB1RCtIp-i8tj1i_pVTxf3baFmnBWD0JmDlCY/edit?usp=sharing")
worksheet = spreadsheet.worksheet("spravochnik_tagging_evaluation.csv")  # Замените на фактическое название листа

# Получаем данные из столбца 'file_contents'
file_contents = worksheet.col_values(5)  # Замените на номер столбца с данными

# Функция для удаления строк с учетом символа "§" и "↑ Cодержание ↑"
def clean_content_with_regex(text):
    # Удаляем все строки до первой строки, которая начинается с символа "§", включая все такие строки, если они идут подряд
    text = re.sub(r'(?s)^.*(?:\n|^)§\d*\s*', '', text)

    # Удаляем все после строки, содержащей "↑ Cодержание ↑", включая её
    text = re.sub(r'↑ Cодержание ↑.*$', '', text, flags=re.DOTALL)

    return text.strip()

# Применяем функцию ко всем ячейкам и собираем обновленные данные
updated_contents = [clean_content_with_regex(content) for content in file_contents]

# Подготавливаем данные для batch_update
cell_updates = []
for i, cleaned_content in enumerate(updated_contents, start=1):
    cell_address = f'F{i}'  # Генерация адреса ячейки в формате A1 (для столбца F)
    cell_updates.append({
        'range': cell_address,
        'values': [[cleaned_content]]  # Двойной список для правильного формата batch_update
    })

# Выполняем обновление данных
worksheet.batch_update(cell_updates)

print("Обработка завершена.")

Обработка завершена.
